In [1]:
import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import os
import pickle
import numpy as np
import nltk
from PIL import Image
import sys
sys.path.append('.')

from pycocotools.coco import COCO

In [2]:


class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [13]:
root = '/data/aihub/Training/images'
label_file ='/data/aihub/Training/labels/vqa_json.json'
vocab = Vocabulary() 

In [14]:
import json
import copy

In [15]:
with open(label_file) as f:
    hub_data = json.load(f)


In [6]:
hub_data['annotations'][0].keys()

dict_keys(['IMAGE_ID', 'IMAGE_NAME', 'KOREAN_IMAGE', 'DIRECT_IMAGE', 'ACTION', 'CAPTION', 'IMAGE_ACQUISITION_DATE', 'IMAGE_URL', 'LICENSE', 'MAINOBJECT', 'MAINOBJECT_URL', 'SCENE', 'SUBOBJECT_1', 'SUBOBJECT_2', 'WIDTH', 'HEIGHT', 'WIKI_FILE', 'bounding_box', 'questions'])

In [7]:
sample = hub_data['annotations'][1]
for k in sample:
    print(k, sample[k])

IMAGE_ID 110200720220621132140
IMAGE_NAME 110200720220621132140.jpg
KOREAN_IMAGE N
DIRECT_IMAGE N
ACTION Watch
CAPTION 두 명의 사람이 빵집에 진열된 프레첼 앞에 서 있다.
IMAGE_ACQUISITION_DATE 20151129
IMAGE_URL https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Pretzels_-_Cologne_Christmas_markets_-_DSC09715.jpg/800px-Pretzels_-_Cologne_Christmas_markets_-_DSC09715.jpg
LICENSE FREE
MAINOBJECT 프레첼
MAINOBJECT_URL http://ko.dbpedia.org/resource/프레첼
SCENE bakery - shop
SUBOBJECT_1 기둥
SUBOBJECT_2 여자
WIDTH 800
HEIGHT 567
WIKI_FILE Pretzels - Cologne Christmas markets - DSC09715.jpg
bounding_box [{'BOX_ID': '119751339', 'OBJECT': '프레첼', 'X_COORDINATE': 317, 'Y_COORDINATE': 359, 'BOX_WIDTH': 54, 'BOX_HEIGHT': 36}]
questions [{'question_id': '110200720220621132140-1', 'visual_concept': ['Object'], 'question_ko': '이미지 속 양과자는 어느 나라 요리에 속해', 'question_en': "The Western Confectionery in the image belongs to a certain country's cuisine", 'answer_ko': '독일', 'answer_en': 'Germany', 'question_type': ['subject', 'su

In [20]:
new_dict = {
    'info':{},
    'licenses':[{
        'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/',
        'id': 1,
        'name': 'Attribution-NonCommercial-ShareAlike License'
        }],
    'images':[],
    'annotations':[]
}
image_dict = {
    'license': 0,
    'file_name': '',#'000000397133.jpg',
    'coco_url': '', #'http://images.cocodataset.org/val2017/000000397133.jpg',
    'height': 0, #427,
    'width': 0, #640,
    'date_captured': '', #'2013-11-14 17:02:52',
    'flickr_url': '',#'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg',
    'id': 0 #397133
    }
ann_dict = {
    'image_id': 0, #179765,
    'id': 0, #38,
    'caption': '',# 'A black Honda motorcycle parked in front of a garage.'
    }

In [21]:
idx = 0
for i, ann in enumerate(hub_data['annotations']):
    if os.path.exists(os.path.join(root, ann['IMAGE_NAME'])):
    # print(ann)
    # break
    # section for img dict
        new_img = copy.deepcopy(image_dict)
        new_ann = copy.deepcopy(ann_dict)
        
        new_img['file_name'] = ann['IMAGE_NAME']
        new_img['height'] = ann['HEIGHT']
        new_img['width'] = ann['WIDTH']
        new_img['id'] = int(ann['IMAGE_ID'])
        
        # section for ann dict
            
        new_ann['image_id'] = int(ann['IMAGE_ID'])
        new_ann['id'] = i
        new_ann['caption'] = ann['CAPTION']
        
        new_dict['images'].append(new_img)
        new_dict['annotations'].append(new_ann)
        idx +=1
    

In [18]:
root

'/data/aihub/Training/images'

In [19]:
os.path.exists(os.path.join(root, hub_data['annotations'][100]['IMAGE_NAME']))


True

In [22]:
with open('/data/aihub/Training/labels/labels.json', 'w') as f:
    json.dump(new_dict, f)

In [38]:
err_list= []
for img in new_dict['images']:
    path = os.path.join('/data/aihub/Training/images/', img['file_name'])
    if not os.path.exists(path):
        print(img)
        err_list.append(img)
        # break

In [27]:
print(len(err_list))

12017


In [29]:
len(hub_data['annotations'])

60084

In [49]:
coco =COCO('/data/aihub/data/Training/labels/TL/labels.json')

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [51]:
coco.imgs

{'110100720220620114213': {'license': 0,
  'file_name': '110100720220620114213.jpg',
  'coco_url': '',
  'height': '640',
  'width': '1134',
  'date_captured': '',
  'flickr_url': '',
  'id': '110100720220620114213'},
 '110200720220621132140': {'license': 0,
  'file_name': '110200720220621132140.jpg',
  'coco_url': '',
  'height': '567',
  'width': '800',
  'date_captured': '',
  'flickr_url': '',
  'id': '110200720220621132140'},
 '110500320220622150258': {'license': 0,
  'file_name': '110500320220622150258.jpg',
  'coco_url': '',
  'height': '673',
  'width': '1280',
  'date_captured': '',
  'flickr_url': '',
  'id': '110500320220622150258'},
 '110400120220621174731': {'license': 0,
  'file_name': '110400120220621174731.jpg',
  'coco_url': '',
  'height': '578',
  'width': '800',
  'date_captured': '',
  'flickr_url': '',
  'id': '110400120220621174731'},
 '110200820220621142828': {'license': 0,
  'file_name': '110200820220621142828.jpg',
  'coco_url': '',
  'height': '576',
  'width

In [11]:


class AIHub_data(data.Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self, root, json, vocab, transform=None):
        """Set the path for images, captions and vocabulary wrapper.
        
        Args:
            root: image directory.
            json: coco annotation file path.
            vocab: vocabulary wrapper.
            transform: image transformer.
        """
        self.root = root
        self.coco = COCO(json)
        self.ids = list(self.coco.anns.keys())
        self.vocab = vocab
        self.transform = transform

    def __getitem__(self, index):
        """Returns one data pair (image and caption)."""
        coco = self.coco
        vocab = self.vocab
        ann_id = self.ids[index]
        caption = coco.anns[ann_id]['caption']
        img_id = coco.anns[ann_id]['image_id']
        path = coco.loadImgs(img_id)[0]['file_name']

        image = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        # Convert caption (string) to word ids.
        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target = torch.Tensor(caption)
        return image, target

    def __len__(self):
        return len(self.ids)

In [12]:
dataset = AIHub_data(
    root='/data/aihub/data/Training/images/',
    json='/data/aihub/data/Training/labels/TL/labels.json',
    vocab=Vocabulary(),
    )

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


[60083]